In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming 'nfile' is the DataFrame from the combined .txt files
# Example data structure
# nfile = pd.read_csv("combined_output.csv")

# Define intervals for 'Peak Field on Sample [mT]'
intervals = [
    {"center": 20, "tolerance": 3},
    {"center": 30, "tolerance": 3},
    {"center": 40, "tolerance": 5},
    {"center": 50, "tolerance": 5}
]

results = []

# Calculate mean and standard deviation for each interval
for interval in intervals:
    lower_bound = interval["center"] - interval["tolerance"]
    upper_bound = interval["center"] + interval["tolerance"]
    
    # Filter data within the interval
    filtered_data = nfile[
        (nfile["Peak Field on Sample [mT]"] >= lower_bound) & 
        (nfile["Peak Field on Sample [mT]"] <= upper_bound)
    ]
    
    # Calculate mean and standard deviation
    mean_resistance = filtered_data["Surface Resistance [nOhm]"].mean()
    std_resistance = filtered_data["Surface Resistance [nOhm]"].std()
    
    # Append results
    results.append({
        "Interval": f"{interval['center']}±{interval['tolerance']}",
        "Mean Resistance (nOhm)": mean_resistance,
        "Std Resistance (nOhm)": std_resistance
    })

# Create a new DataFrame for results
results_df = pd.DataFrame(results)

# Save the results to a CSV file
results_df.to_csv("resistance_analysis.csv", index=False)

# Print the results
print(results_df)

# Plot the results
plt.errorbar(
    results_df["Interval"],
    results_df["Mean Resistance (nOhm)"],
    yerr=results_df["Std Resistance (nOhm)"],
    fmt='o',
    capsize=5,
    label="Surface Resistance"
)

plt.title("Surface Resistance Analysis")
plt.xlabel("Peak Field Interval [mT]")
plt.ylabel("Surface Resistance [nOhm]")
plt.grid(True)
plt.legend()
plt.savefig("resistance_analysis_plot.pdf", format="pdf", bbox_inches="tight")
plt.show()


In [ ]:
from funclib.importf import *

script_dir = os.path.dirname(os.path.abspath("main.ipynb"))
phd_dir = os.path.dirname(script_dir)
TestPath = os.path.join(phd_dir, "QPR Data", "2022-04-04 - test #36 - ARIES B-3.19 Siegen SIS")

Test1 = HandleTest(TestPath, debug = True)
Test1Data = Test1.load_data(pattern="*41*MHz*.txt")

#pd.set_option('display.max_rows', 100)
Test1.data.info()

#Test1Data.plot.scatter(x="Peak Field on Sample [mT]", y="Surface Resistance [nOhm]")
#plt.show()


In [ ]:
from funclib.importf import *

script_dir = os.path.dirname(os.path.abspath("main.ipynb"))
phd_dir = os.path.dirname(script_dir)
TestPath = os.path.join(phd_dir, "QPR Data", "2022-04-04 - test #36 - ARIES B-3.19 Siegen SIS")
print(TestPath)

Test1 = HandleTest(TestPath)
Test1Data = Test1.load_data(pattern="*41*MHz*.txt")

#Test1Data.info()
#pd.set_option('display.max_rows', None)
#print(Test1Data["File Name"])
#print(Test1.Data["Run"].unique())

Test1.plot_histogram(step = 0.5, ParamName=FieldNames.SENS_B, ParamVal=2.5, ParamTol=0.05)


In [ ]:
import pandas as pd
import numpy as np

# Create DataFrame
data = {
    "A": [f"A{i}" for i in range(1, 11)],  # A1, A2, ..., A10
    "B": [np.random.choice(range(1, 10), 5, replace=False).tolist() for _ in range(10)]  # Random lists in B
}

df = pd.DataFrame(data)
print("Original DataFrame:")
print(df)

print(df.loc[6,"B"])
print(type(df.loc[6,"B"]))

print(df.loc[df.loc[6,"B"],"A"])


In [28]:
import pandas as pd
import numpy as np
import statistics as stat

# Create DataFrame
data = {
    "A": [i for i in range(1, 11)]+[12,15,16,31,32,33,34,41,42,43],  # A1, A2, ..., A10
    "B": [np.random.choice(range(1, 10), 5, replace=False).tolist() for _ in range(20)]  # Random lists in B
}

print(len(data["A"]))
print(len(data["B"]))

x_axis="A"

dataset = pd.DataFrame(data)
print("Original DataFrame:")
print(dataset)

print(dataset.loc[1,x_axis])


index = 0
res = 1

avg_list = []
meanlist = []
stdlist = []
while index < dataset.index.max():
    if abs(dataset.loc[index, x_axis]-dataset.loc[index+1, x_axis])<=res:
        avg_list.append(dataset.loc[index, x_axis])
    else:
        meanlist.append(np.mean(avg_list))
        stdlist.append(np.std(avg_list))
        avg_list = []
    index += 1
    print(index)

print(f"Mean: {meanlist}")

# Calculate standard deviation

print(f"Standard Deviation: {stdlist}")


20
20
Original DataFrame:
     A                B
0    1  [6, 4, 3, 2, 7]
1    2  [2, 1, 5, 8, 7]
2    3  [5, 3, 1, 4, 2]
3    4  [7, 5, 9, 2, 3]
4    5  [6, 9, 7, 1, 3]
5    6  [9, 3, 7, 2, 6]
6    7  [3, 2, 8, 6, 1]
7    8  [8, 7, 2, 5, 4]
8    9  [9, 7, 3, 4, 1]
9   10  [4, 1, 9, 3, 8]
10  12  [2, 7, 5, 4, 3]
11  15  [7, 3, 2, 1, 4]
12  16  [5, 4, 1, 9, 2]
13  31  [5, 6, 2, 1, 4]
14  32  [5, 3, 7, 8, 4]
15  33  [8, 1, 7, 5, 3]
16  34  [8, 2, 7, 1, 9]
17  41  [6, 4, 8, 2, 7]
18  42  [9, 7, 8, 6, 1]
19  43  [7, 4, 3, 9, 2]
2
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
Mean: [5.0, nan, 15.0, 32.0]
Standard Deviation: [2.581988897471611, nan, 0.0, 0.816496580927726]


/home/dmitry/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/dmitry/.local/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/dmitry/.local/lib/python3.10/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/dmitry/.local/lib/python3.10/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/home/dmitry/.local/lib/python3.10/site-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
